<a href="https://colab.research.google.com/github/litlsun/compling/blob/main/%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_4_SMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Statistical Machine Translation
**SMT (Statistical Machine Translation)** — это статистический подход к машинному переводу, который использует вероятностные модели для нахождения наиболее правдоподобного перевода предложения с одного языка (X) на другой (Y).

## Основные термины SMT

1. Статистический машинный перевод (SMT):
   - Подход к машинному переводу, основанный на статистических методах и вероятностных моделях для нахождения наиболее вероятного перевода.

2. Языковая модель (Language Model):
   - Модель, которая оценивает вероятность последовательностей слов в целевом языке (Y). Использует n-граммы для определения вероятности появления слова на основе предыдущих слов.

3. Модель перевода (Translation Model):
   - Модель, которая определяет соответствия между фразами исходного языка (X) и целевого языка (Y). Она вычисляет вероятность *P(x|y)*, где *x* — фраза на языке X, а *y* — её перевод на язык Y.

4. Параллельный корпус (Parallel Corpus):
   - Набор текстов, где каждое предложение на одном языке имеет соответствующее предложение на другом языке. Используется для обучения моделей перевода.

5. Выравнивание фраз (Phrase Alignment):
   - Процесс сопоставления фраз в параллельном корпусе для определения их соответствий и вероятностей.

6. N-грамма (N-gram):
   - Последовательность из N элементов (слов или символов), используемая для оценки вероятности появления слов в языковой модели.

7. Декодер (Decoder):
   - Алгоритм, который ищет наиболее грамматически правильные и лексически подходящие переводы, выбирая лучший перевод среди всех возможных гипотез.

8. Вероятностные модели (Probabilistic Models):
   - Модели, использующие теорию вероятностей для оценки различных аспектов перевода.

9. Гипотеза перевода (Translation Hypothesis):
   - Возможный перевод исходного текста, который предлагается моделью для оценки его вероятности.

10. BLEU-метрика (Bilingual Evaluation Understudy):
    - Автоматическая метрика для оценки качества машинного перевода путем сравнения с референсными переводами.

## Основные этапы SMT:

### 1. Сбор и подготовка данных
- Параллельный корпус: Сбор текстов на исходном (X) и целевом (Y) языках, где каждое предложение на одном языке имеет соответствующее предложение на другом. Это основа для обучения моделей.
- Выравнивание фраз: Процесс сопоставления фраз в параллельном корпусе для определения их соответствий и вероятностей.

### 2. Обучение моделей
- Языковая модель: Обучение модели, которая оценивает вероятность последовательностей слов в целевом языке (Y). Обычно используется n-граммный подход.
- Модель перевода: Обучение модели, которая определяет соответствия между фразами исходного языка (X) и целевого языка (Y). Вычисляет вероятность *P(x|y)*.

### 3. Генерация кандидатов перевода
- Создание гипотез перевода: Для каждого входного предложения *x* генерируются все возможные кандидаты перевода *y* на основе моделей, обученных на предыдущих этапах.

### 4. Декодирование
- Поиск лучшего перевода: Используя декодер, выбирается наиболее вероятный перевод среди всех сгенерированных гипотез. Это происходит путем максимизации функции вероятности *P(y|x)*.

### 5. Оценка качества перевода
- Метрики оценки: Использование различных метрик, таких как BLEU, для оценки качества перевода по сравнению с референсными переводами.

### 6. Постобработка
- Коррекция и улучшение: После получения перевода может потребоваться дополнительная обработка для улучшения его грамматической структуры и лексической точности.



In [1]:
import tarfile

from sklearn.model_selection import train_test_split

from collections import Counter, defaultdict
import random

In [3]:
# загрузим датасет 292 short sentences и извлечем файлы из архива
with tarfile.open('de-en.tiny.tgz', 'r:gz') as tar:
  tar.extractall()

In [6]:
!ls

de-en.tiny.de  de-en.tiny.en  de-en.tiny.tgz  sample_data


In [8]:
with open('de-en.tiny.de', 'r') as f:
  german = f.read().split('\n')[:-1]

with open('de-en.tiny.en', 'r') as f:
  english = f.read().split('\n')[:-1]

print("Данные языка X:\n", german)
print("Данные языка Y:\n", english)

Данные языка X:
 ['Wiederaufnahme der Sitzungsperiode', 'Ich bitte Sie , sich zu einer Schweigeminute zu erheben .', '( Das Parlament erhebt sich zu einer Schweigeminute . )', 'Frau Präsidentin , zur Geschäftsordnung .', 'Wenn das Haus damit einverstanden ist , werde ich dem Vorschlag von Herrn Evans folgen .', 'Frau Präsidentin , zur Geschäftsordnung .', 'Könnten Sie mir eine Auskunft zu Artikel 143 im Zusammenhang mit der Unzulässigkeit geben ?', 'Und zwar sollen derartige Strafen trotz des Grundsatzes der relativen Stabilität verhängt werden .', 'All dies entspricht den Grundsätzen , die wir stets verteidigt haben .', 'Vielen Dank , Herr Segni , das will ich gerne tun .', 'Das ist ganz im Sinne der Position , die wir als Parlament immer vertreten haben .', 'Das ist der Fall von Alexander Nikitin .', 'Nun ist es aber so , daß er wieder angeklagt werden soll , weil der Staatsanwalt in Berufung geht .', 'Dennoch , Frau Präsidentin , wurde meinem Wunsch nicht entsprochen .', 'Deshalb mö

In [9]:
X_train, X_test, y_train, y_test = train_test_split(english, german)

print("> Обучающая выборка:")
for text, label in zip(X_train, y_train):
    print(f"\nТекст на немецком: {label}\n Его перевод на английский: {text}\n")

print("> Тестовая выборка:")
for text, label in zip(X_test, y_test):
    print(f"\nТекст на немецком: {label}\n Его перевод на английский: {text}\n")

> Обучающая выборка:

Текст на немецком: Die eigentliche Arbeit ist getan , nun geht es um die Umsetzung .
 Его перевод на английский: The work is done . All that remains is the business of enforcement .


Текст на немецком: Bitte nicht noch einen weiteren Bereich , in dem wir dann anschließend das Vollzugsdefizit beklagen müssen .
 Его перевод на английский: Please let this not be yet another sector where we subsequently have to lament the lack of enforcement .


Текст на немецком: Letztlich können Beihilfen dazu dienen , Umstrukturierungen zu ermöglichen , Schulungen anzubieten , Arbeitsplätze und damit Know-how zu retten .
 Его перевод на английский: Finally , aid can enable restructuring , offer training , save jobs and thus know-how .


Текст на немецком: Ich möchte zwei davon kurz zusammenfassen , der Berichterstatter hat sie schon aufgegriffen , eine positive und eine negative .
 Его перевод на английский: I shall summarise two of those highlighted by the rapporteur , one positi

In [10]:
def tokenize(sentences):
  return [sentence.split() for sentence in sentences]
X_train_tokens, X_test_tokens, y_train_tokens, y_test_tokens = tokenize(X_train), tokenize(X_test), tokenize(y_train), tokenize(y_test)

print('Образец токенизированного текста:', X_train_tokens)

Образец токенизированного текста: [['The', 'work', 'is', 'done', '.', 'All', 'that', 'remains', 'is', 'the', 'business', 'of', 'enforcement', '.'], ['Please', 'let', 'this', 'not', 'be', 'yet', 'another', 'sector', 'where', 'we', 'subsequently', 'have', 'to', 'lament', 'the', 'lack', 'of', 'enforcement', '.'], ['Finally', ',', 'aid', 'can', 'enable', 'restructuring', ',', 'offer', 'training', ',', 'save', 'jobs', 'and', 'thus', 'know-how', '.'], ['I', 'shall', 'summarise', 'two', 'of', 'those', 'highlighted', 'by', 'the', 'rapporteur', ',', 'one', 'positive', 'and', 'one', 'negative', '.'], ['Under', 'Article', '88', ',', 'the', 'Commission', 'is', 'obliged', 'to', 'supervise', 'state', 'aid', '.'], ['I', 'would', 'like', 'to', 'explain', 'our', 'thinking', 'here', '.'], ['It', 'is', 'one', 'of', 'the', 'steps', 'towards', 'a', 'policy', 'of', 'employment', 'and', 'sustainable', 'development', '.'], ['That', 'was', 'the', 'decision', '.'], ['The', 'report', 'is', 'hereby', 'withdrawn',

In [11]:
x_vocab = Counter(' '.join(german).split()).keys()
y_vocab = Counter(' '.join(english).split()).keys()

print(f"Словарь немецких словоформ: {x_vocab}\n Всего {len(x_vocab)} словоформ")
print(f"\nCловарь английских словоформ: {y_vocab}\n Всего {len(y_vocab)} словоформ")

Словарь немецких словоформ: dict_keys(['Wiederaufnahme', 'der', 'Sitzungsperiode', 'Ich', 'bitte', 'Sie', ',', 'sich', 'zu', 'einer', 'Schweigeminute', 'erheben', '.', '(', 'Das', 'Parlament', 'erhebt', ')', 'Frau', 'Präsidentin', 'zur', 'Geschäftsordnung', 'Wenn', 'das', 'Haus', 'damit', 'einverstanden', 'ist', 'werde', 'ich', 'dem', 'Vorschlag', 'von', 'Herrn', 'Evans', 'folgen', 'Könnten', 'mir', 'eine', 'Auskunft', 'Artikel', '143', 'im', 'Zusammenhang', 'mit', 'Unzulässigkeit', 'geben', '?', 'Und', 'zwar', 'sollen', 'derartige', 'Strafen', 'trotz', 'des', 'Grundsatzes', 'relativen', 'Stabilität', 'verhängt', 'werden', 'All', 'dies', 'entspricht', 'den', 'Grundsätzen', 'die', 'wir', 'stets', 'verteidigt', 'haben', 'Vielen', 'Dank', 'Herr', 'Segni', 'will', 'gerne', 'tun', 'ganz', 'Sinne', 'Position', 'als', 'immer', 'vertreten', 'Fall', 'Alexander', 'Nikitin', 'Nun', 'es', 'aber', 'so', 'daß', 'er', 'wieder', 'angeklagt', 'soll', 'weil', 'Staatsanwalt', 'in', 'Berufung', 'geht', 'D

In [12]:
# вероятность того, что случайное слово x_vocab соответсвует случайному слову y_vocab
uniform = 1 / (len(x_vocab) * len(y_vocab))

round(uniform, 3)

0.0

In [13]:
# t-model
t = {}

for i in range(len(X_train)):
  # начинаем итерацию по обучающей выборке
  for word_x in X_train_tokens[i]:
    for word_y in y_train_tokens[i]:
      # создаем t-table
      t[(word_x, word_y)] = uniform

# t-table
for elem in t:
  print("Соответствие |", elem[0], "  ->  ", elem[1], "| Вероятность:", round(t[elem], 3))

Выходные данные были обрезаны до нескольких последних строк (5000).
Соответствие | link   ->   Schaffung | Вероятность: 0.0
Соответствие | link   ->   von | Вероятность: 0.0
Соответствие | link   ->   Arbeitsplätzen | Вероятность: 0.0
Соответствие | link   ->   zu | Вероятность: 0.0
Соответствие | link   ->   binden | Вероятность: 0.0
Соответствие | link   ->   . | Вероятность: 0.0
Соответствие | structural   ->   Deshalb | Вероятность: 0.0
Соответствие | structural   ->   ist | Вероятность: 0.0
Соответствие | structural   ->   es | Вероятность: 0.0
Соответствие | structural   ->   erforderlich | Вероятность: 0.0
Соответствие | structural   ->   , | Вероятность: 0.0
Соответствие | structural   ->   strukturelle | Вероятность: 0.0
Соответствие | structural   ->   Finanzierung | Вероятность: 0.0
Соответствие | structural   ->   stärker | Вероятность: 0.0
Соответствие | structural   ->   an | Вероятность: 0.0
Соответствие | structural   ->   Schaffung | Вероятность: 0.0
Соответствие | str

In [17]:
# количество итераций обучения
epochs = 1000

In [18]:
for epoch in range(epochs):
  # начинаем обучение

  # шаг 0. создаем слоты для подсчета статистики
  count = {} # P(x|y)
  total = {} # P(y)

  for i in range(len(X_train)):
    # начинаем итерацию по обучающей выборке
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        # создаем слоты для подсчета условной вероятности совпадений в корпусе
        count[(word_x, word_y)] = 0
        # и слоты для статистической языковой модели y
        total[word_y] = 0

  # шаг 1. Expectation
  for i in range(len(X_train)):
    # начинаем итерацию по обучающей выборке
    total_stat = {} # статистика x

    # собираем предварительную статистику на основе данных x
    for word_x in X_train_tokens[i]:
      total_stat[word_x] = 0 # создаем слоты для подсчета статистики по каждому токену x
      for word_y in y_train_tokens[i]:
        # обновляем данные из t-table; увеличиваем значения при обнаружении совместной встречаемости
        total_stat[word_x] += t[(word_x, word_y)]

    # обновляем данные для P(x|y) и P(y)
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        # подсчет условной вероятности совпадений в корпусе: равномерное распределение / частотность x
        count[(word_x, word_y)] += t[(word_x, word_y)] / total_stat[word_x]
        # подсчет статистической информации y: равномерное распределение / частотность x
        total[word_y] += t[(word_x, word_y)] / total_stat[word_x]

  # шаг 2. Maximization
  for i in range(len(X_train)):
    # начинаем итерацию по обучающей выборке
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        # обновляем t-table: вероятность совпадения в корпусе / вероятность информации y
        t[(word_x, word_y)] = count[(word_x, word_y)] / total[word_y]

for elem in t:
  print("Соответствие |", elem[0], "  ->  ", elem[1], "| Вероятность:", round(t[elem], 3))

Выходные данные были обрезаны до нескольких последних строк (5000).
Соответствие | link   ->   Schaffung | Вероятность: 0.0
Соответствие | link   ->   von | Вероятность: 0.0
Соответствие | link   ->   Arbeitsplätzen | Вероятность: 0.0
Соответствие | link   ->   zu | Вероятность: 0.0
Соответствие | link   ->   binden | Вероятность: 0.145
Соответствие | link   ->   . | Вероятность: 0.0
Соответствие | structural   ->   Deshalb | Вероятность: 0.0
Соответствие | structural   ->   ist | Вероятность: 0.0
Соответствие | structural   ->   es | Вероятность: 0.0
Соответствие | structural   ->   erforderlich | Вероятность: 0.0
Соответствие | structural   ->   , | Вероятность: 0.0
Соответствие | structural   ->   strukturelle | Вероятность: 0.145
Соответствие | structural   ->   Finanzierung | Вероятность: 0.145
Соответствие | structural   ->   stärker | Вероятность: 0.145
Соответствие | structural   ->   an | Вероятность: 0.0
Соответствие | structural   ->   Schaffung | Вероятность: 0.0
Соответств

In [19]:
# для обучения модели объединим 2 выборки
tokens = ' '.join(german).split()

# хранилище для биграмм
bigram_model = defaultdict(list)

# собираем все попарные совпадения
for i in range(len(tokens)-1):
    current_word = tokens[i]
    next_word = tokens[i + 1]
    bigram_model[current_word].append(next_word)

print(bigram_model)

def decoder(model, steps=5):
  # инициализация случайного токена
  current_word = random.choice(tokens)
  generated_sentence = current_word

  for step in range(steps):
    # пошаговая генерация
    print('Шаг', step+1)
    next_word_options = model[current_word]
    print(f'Правдоподобные варианты продолжения для токена {current_word}:', next_word_options)

    current_word = random.choice(next_word_options)
    generated_sentence += ' '
    generated_sentence += current_word
    print('Промежуточный результат:', generated_sentence)
    print()
  print('Результат:', generated_sentence)

decoder(bigram_model)

defaultdict(<class 'list'>, {'Wiederaufnahme': ['der'], 'der': ['Sitzungsperiode', 'Unzulässigkeit', 'relativen', 'Position', 'Fall', 'Staatsanwalt', 'Arbeitsschutzausschuß', 'Fall', 'dortigen', 'großen', 'EU-Mitgliedstaaten', 'EU', 'Lage', 'PSE-Fraktion', 'Konferenz', 'Präsidenten', 'Konferenz', 'Präsidenten', 'Beschluß', 'Sozialistischen', 'PPE-DE-Fraktion', 'Tagesordnung', 'Fraktion', 'Tagesordnung', 'gegen', 'Werbetrommel', 'Abstimmung', 'Sitzung', 'Subsidiarität', 'ersten', 'Verkehrssicherheit', 'Prüfungsvorschriften', 'Kommission', 'Straße', 'Anwendung', 'technischen', 'Transportbehälter', 'Kommission', 'morgigen', 'Kommission', 'Zusätzlichkeit', 'Kommission', 'Wirtschaftsstruktur', 'Sozialwirtschaft', 'Arbeitsplätze', 'Union', 'fragwürdigen', 'Konservativen', 'entsprechenden', 'Leitlinien', 'Erarbeitung', 'Leitlinien', 'Evaluierung', 'Ergebnisse', 'Bericht', 'Städte', 'nationalen', 'regionalen', 'Mittel', 'Politik', 'Mittel', 'Mittelauszahlung', 'Mitgliedstaaten', 'Leitlinien', 

In [21]:
# сортировка t-table по убыванию правдоподобия
sorted_t = sorted(t.items(), key=lambda k: (k[1], k[0]), reverse=True)

def translate(token):
    for element in sorted_t:
        if element[0][1] == token:
            # поиск совпадений в t-table
            return element[0][0]
    return None  # Возвращаем None, если совпадений не найдено

for sentence in y_test_tokens:
    print("Оригинальное предложение:", ' '.join(sentence))
    translation = list(map(lambda token: translate(token) or '', sentence))  # Заменяем None на ''
    print("Перевод:", ' '.join(translation))

Оригинальное предложение: Ich will zum Ausdruck bringen , daß die Sicherheit ein vorrangiges Ziel der Kommission ist .
Перевод: I . these very  , that the safety a  these the Commission is .
Оригинальное предложение: Ich möchte nochmals allen Rednern und ganz besonders dem Berichterstatter , Herrn Koch , meinen Dank sagen .
Перевод: I would well have  and quite growing the rapporteur , what  , saying Thank in .
Оригинальное предложение: Sozio-ökonomische Lage und Entwicklung der Regionen
Перевод:  to and to the regions
Оригинальное предложение: Seine effektive Umsetzung bedarf einer starken Verankerung in den nationalen Behörden der Mitgliedstaaten .
Перевод:   of  ,   in the   the Member .
Оригинальное предложение: Ich schlage vor , daß sich Herr Evans nochmals mit der entsprechenden Verordnung vertraut macht .
Перевод: I  to , that and Mr suggested well with the  text   .
Оригинальное предложение: Es geht um den Anspruch der Modernisierung , um die Zukunftstauglichkeit der europäisch

In [25]:
from nltk.translate.bleu_score import corpus_bleu

# Используем только одно референсное предложение
reference = [X_test_tokens[0]]  # Или любое другое предложение

# Создаем candidate для одного предложения
candidate = [translate(token) for token in y_test_tokens[0]]

# Теперь reference и candidate имеют одинаковую длину
bleu_score = corpus_bleu([reference], [candidate])  # Обернуть в список

print("BLEU Score:", bleu_score)


BLEU Score: 6.373704167435469e-155


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [26]:
reference

[['But',
  'I',
  'would',
  'like',
  'to',
  'say',
  'that',
  'safety',
  'is',
  'a',
  'priority',
  'objective',
  'for',
  'the',
  'Commission',
  '.']]

In [27]:
candidate

['I',
 '.',
 'these',
 'very',
 None,
 ',',
 'that',
 'the',
 'safety',
 'a',
 None,
 'these',
 'the',
 'Commission',
 'is',
 '.']